<a href="https://colab.research.google.com/github/suhyeon0325/BELKA-DrugPredict/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 42.2 MB/s eta 0:00:00


In [3]:
# 데이터 처리 및 분석
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 분자 데이터 처리
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw, Crippen, rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import GraphDescriptors
from rdkit.Chem import rdchem
from rdkit.Chem import EnumerateStereoisomers
from rdkit.Chem import ResonanceMolSupplier

# 머신러닝
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.manifold import TSNE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# 딥러닝
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D

# 기타 유틸리티
import duckdb
import os
import sys
import json
import warnings
from collections import defaultdict

# 경고 메시지 무시 설정
warnings.filterwarnings('ignore')

In [4]:
train_path = '/content/drive/MyDrive/BELKA-DrugPredict/data/train.parquet'
con = duckdb. connect()

In [5]:
train_path = '/content/drive/MyDrive/BELKA-DrugPredict/data/train.parquet'
con = duckdb.connect()

# 각 타겟 단백질에 대해 샘플 추출
train = con.query(f"""
(SELECT *
 FROM parquet_scan('{train_path}')
 WHERE binds = 0 AND protein_name = 'EPHX2'
 ORDER BY random()
 LIMIT 30000)
UNION ALL
(SELECT *
 FROM parquet_scan('{train_path}')
 WHERE binds = 1 AND protein_name = 'EPHX2'
 ORDER BY random()
 LIMIT 3000)
UNION ALL
(SELECT *
 FROM parquet_scan('{train_path}')
 WHERE binds = 0 AND protein_name = 'BRD4'
 ORDER BY random()
 LIMIT 30000)
UNION ALL
(SELECT *
 FROM parquet_scan('{train_path}')
 WHERE binds = 1 AND protein_name = 'BRD4'
 ORDER BY random()
 LIMIT 3000)
UNION ALL
(SELECT *
 FROM parquet_scan('{train_path}')
 WHERE binds = 0 AND protein_name = 'HSA'
 ORDER BY random()
 LIMIT 30000)
UNION ALL
(SELECT *
 FROM parquet_scan('{train_path}')
 WHERE binds = 1 AND protein_name = 'HSA'
 ORDER BY random()
 LIMIT 3000)
""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0


## Feature Engineering

### 분자 무게 변수 추가

In [7]:
# 분자 무게를 계산하고 데이터프레임에 추가하는 함수
def add_molecular_weight(df):
    # 분자 무게 계산을 위한 내부 함수
    def calculate_mw(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return Descriptors.MolWt(molecule) if molecule else None

    # 각 building block과 molecule의 분자 무게를 계산하여 새로운 열로 추가
    df['mw_buildingblock1'] = df['buildingblock1_smiles'].apply(calculate_mw)
    df['mw_buildingblock2'] = df['buildingblock2_smiles'].apply(calculate_mw)
    df['mw_buildingblock3'] = df['buildingblock3_smiles'].apply(calculate_mw)
    df['mw_molecule'] = df['molecule_smiles'].apply(calculate_mw)

    return df

# 'train' 데이터프레임에 분자 무게 추가
train = add_molecular_weight(train)

In [8]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980


### LogP 변수 추가
- 로그 P 값이 높은 분자는 일반적으로 더 소수성이며, 이는 약물이 세포막을 통과할 가능성이 높음을 의미 -> 이러한 성질의 약물의 생체 이용률과 직접적 관련이 있음
- 친수성(로그 P값이 낮은) 분자는 일반적으로 물에 더 잘 용해되며, 이는 약물의 안정성 및 투여 시 용해도에 영향
- 신약 개발 과정에서 LogP 값은 중요한 최적화 대상 -> 약물 후보물질이 너무 소수성이거나 너무 친수성인 경우 부작용을 초래하거나 원치 않는 약물 동태학적 특성을 나타낼 수 있음
- 약물의 특정 약효 그룹이나 대상 병리학적 상태에 대한 친화성을 분석할 때 로그 P 값이 중요한 역할 (예: 특정 로그 P 값 범위를 가진 분자들이 특정 질병에 대해 더 높은 효과를 보일 수 있음)

In [9]:
def add_logP(df):
    # 로그 P 값을 계산하는 내부 함수
    def calculate_logP(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return Crippen.MolLogP(molecule) if molecule else None

    # molecule SMILES로부터 로그 P 값을 계산하여 새로운 열로 추가
    df['logP'] = df['molecule_smiles'].apply(calculate_logP)

    return df

# 'train' 데이터프레임에 로그 P 값을 추가
train = add_logP(train)

In [10]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule,logP
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036,3.02098
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958,2.97550
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502,2.61982
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996,4.76192
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980,1.33510


### HBD와 HBA 변수 추가
- 수소결합 기증자와 수용체 수는 분자가 세포막을 통과할 수 있는 능력에 영향(물과의 상호작용을 통해 분자의 용해도와 투과성을 결정)

- 수소결합은 약물과 타겟 사이의 결합 강도에 중요한 영향 -> 기증자 및 수용체의 존재는 약물이 타겟 단백질에 결합하는 위치와 방식을 결정 -> 이는 약효의 강도와 지속성에 직접적인 영향

- 약물 후보물질을 최적화하는 과정에서 수소결합 기증자와 수용체의 수를 조절함으로써, 약물의 생체 이용률과 타겟과의 결합 특성을 개선

- 약물의 흡수, 분배, 대사, 배설 및 독성(ADMET) 프로파일을 예측하는 데 있어 수소결합 특성은 중요한 정보 제공

In [11]:
def add_hydrogen_bond_features(df):
    # 수소결합 기증자와 수용체 수를 계산하는 함수
    def calculate_hbd(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return rdMolDescriptors.CalcNumHBD(molecule) if molecule else None

    def calculate_hba(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return rdMolDescriptors.CalcNumHBA(molecule) if molecule else None

    # 수소결합 기증자와 수용체 수를 계산하여 데이터프레임에 추가
    df['num_hbd'] = df['molecule_smiles'].apply(calculate_hbd)
    df['num_hba'] = df['molecule_smiles'].apply(calculate_hba)

    return df

# 'train' 데이터프레임에 수소결합 특성 추가
train = add_hydrogen_bond_features(train)

In [12]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule,logP,num_hbd,num_hba
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036,3.02098,4,13
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958,2.97550,4,9
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502,2.61982,6,11
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996,4.76192,6,10
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980,1.33510,5,10


### 회전 가능한 결합 수
- 회전 가능한 결합수는 분자의 유연성을 나타냄 - > 분자가 특정 형태의 수용체 사이트에 맞춰 구조를 조정할 수 있는 능력과 관련 -> 유연성이 높으면 다양한 형태의 수용체에 더 쉽게 접근하고 결합
- 회전 가능한 결합의 수가 많으면 분자가 더 크고 복잡 -> 일반적으로 세포막 투과성을 감소시킬 수 있음 -> 약물 설계 시 이를 최적화하여 약물의 흡수와 생체 이용률을 개선가능
- 회전 가능한 결합수를 조절함으로써 약물의 구조적 유연성을 조정할 수 있으며, 이는 약물의 타겟 결합 특성과 효능을 최적화 -> 약물이 특정 수용체와의 결합을 위해 필요한 구조적 조정을 할 수 있도록 유도가능
- 회전 가능한 결합의 수는 약물의 대사 및 배설 경로에도 영향 -> 유연성이 높으면 대사 과정에서 다양한 변형을 겪을 수 있음 -. 이는 안전성 평가 및 관리에 중요

In [13]:
def add_rotatable_bonds(df):
    # 회전 가능한 결합수를 계산하는 함수
    def calculate_rot_bonds(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return rdMolDescriptors.CalcNumRotatableBonds(molecule) if molecule else None

    # 회전 가능한 결합수를 계산하여 데이터프레임에 추가
    df['num_rotatable_bonds'] = df['molecule_smiles'].apply(calculate_rot_bonds)

    return df

# 'train' 데이터프레임에 회전 가능한 결합수 특성 추가
train = add_rotatable_bonds(train)

### Ring 개수
- 고리 구조는 분자의 화학적 안정성에 기여 -> 특히 방향족 고리(Aromatic rings)는 화학적으로 매우 안정적이며, 약물 분자 내에서 중요
- 고리 구조는 특정 유형의 화학 반응에 중요 역할 -> 예를 들어, 일부 고리는 전자를 제공하거나 받아들이기 쉬워서 분자간의 상호작용을 촉진 가능
- 고리 구조는 약물이 수용체와 결합할 때 특정 위치에 고정되거나 구조적인 맞춤을 할 수 있게 도움 -> 타겟 분자와의 결합 특성과 선택성 향상
- 고리 수는 약물의 설계 및 최적화 과정에서 중요한 변수. 특히 분자의 크기, 형태 및 용해도에 영향을 미치며, 이는 약물의 생체 이용률 및 약효에 직접적 영향
- 일부 고리 구조는 대사 과정에서 특정 대사효소와 상호작용 가능 -> 약물의 대사 속도 및 대사 부산물의 유형에 영향


In [14]:
def add_ring_count(df):
    # 고리 수를 계산하는 함수
    def calculate_ring_count(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return rdMolDescriptors.CalcNumRings(molecule) if molecule else None

    # 고리 수를 계산하여 데이터프레임에 추가
    df['num_rings'] = df['molecule_smiles'].apply(calculate_ring_count)

    return df

# 'train' 데이터프레임에 RING 수 특성 추가
train = add_ring_count(train)

In [15]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule,logP,num_hbd,num_hba,num_rotatable_bonds,num_rings
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036,3.02098,4,13,10,5
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958,2.97550,4,9,6,4
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502,2.61982,6,11,12,4
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996,4.76192,6,10,10,5
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980,1.33510,5,10,9,3


### Substructure 수
- 특정 부분 구조의 존재와 수는 분자의 화학적 특성과 반응성을 이해하는 데 중요 (예: 벤젠 고리와 같은 방향족 구조는 분자의 화학적 안정성과 UV/Vis 스펙트럼 특성에 영향)
- 특정 부분 구조는 약물이 타겟 단백질과 결합하는 방식에 중요한 영향 -> 특정 부분 구조가 타겟의 결합 부위에 적합하게 배치되어 약물의 효능과 선택성을 높일 수 있음
- 약물 설계 과정에서 부분 구조 분석을 통해 특정 화학적 특성이나 생물학적 활성과의 관계를 파악하고, 이를 바탕으로 더 나은 약물 후보 설계 가능
- 일부 부분 구조는 약물의 대사 경로와 관련이 있으며, 이는 대사 안정성 및 잠재적인 독성 프로파일에 중요한 영향

In [16]:
def add_substructure_count(df, substruct_smiles):
    # 부분 구조를 RDKit 분자 객체로 변환
    substructure = Chem.MolFromSmiles(substruct_smiles)

    # 주어진 분자 내에서 부분 구조의 수를 계산하는 함수
    def calculate_substructure_count(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return len(molecule.GetSubstructMatches(substructure)) if molecule else 0

    # 부분 구조 수를 계산하여 데이터프레임에 추가
    column_name = f'num_{substruct_smiles}'
    df[column_name] = df['molecule_smiles'].apply(calculate_substructure_count)

    return df

# 'train' 데이터프레임에 벤젠 고리 수 특성 추가
train = add_substructure_count(train, 'c1ccccc1')

In [17]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule,logP,num_hbd,num_hba,num_rotatable_bonds,num_rings,num_c1ccccc1
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036,3.02098,4,13,10,5,2
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958,2.97550,4,9,6,4,1
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502,2.61982,6,11,12,4,1
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996,4.76192,6,10,10,5,3
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980,1.33510,5,10,9,3,0


### TPSA(위임 면적 합계, Topological Polar Surface Area)
- TPSA는 분자가 세포막을 통과하는 능력과 밀접한 관계 -> 일반적으로 TPSA가 낮은 분자는 더 높은 세포막 투과성을 가지며, 이는 경구 투여 시 약물의 흡수율이 높아짐을 미
- 극성 표면적은 분자가 수용체와 수소 결합을 형성할 수 있는 능력을 나타냄 -> 분자가 수용체에 얼마나 강하게 또는 효과적으로 결합할 수 있는지 예측
- TPSA는 약물이 혈액-뇌 장벽(Blood-Brain Barrier, BBB)을 통과하는 능력과도 관련 -> 일반적으로 TPSA값이 낮은 약물은 BBB를 통과하기 더 쉬우며, 중추 신경계 치료제로서의 가능성이 높음
- TPSA는 약물의 대사 경로 및 잠재적 독성에 대한 통찰 제공 -> 높은 TPSA 값을 가진 약물은 대사가 더 복잡하거나 예측하기 어려울 수 있음

In [18]:
def add_tpsa(df):
    # TPSA를 계산하는 함수
    def calculate_tpsa(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return rdMolDescriptors.CalcTPSA(molecule) if molecule else 0

    # TPSA를 계산하여 데이터프레임에 추가
    df['tpsa'] = df['molecule_smiles'].apply(calculate_tpsa)

    return df

# 'train' 데이터프레임에 TPSA 특성 추가
train = add_tpsa(train)

In [19]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule,logP,num_hbd,num_hba,num_rotatable_bonds,num_rings,num_c1ccccc1,tpsa
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036,3.02098,4,13,10,5,2,180.48
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958,2.97550,4,9,6,4,1,133.29
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502,2.61982,6,11,12,4,1,191.60
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996,4.76192,6,10,10,5,3,178.71
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980,1.33510,5,10,9,3,0,149.61


### 분자 복잡성
- 분자 복잡성은 약물의 합성 난이도를 예측하는 데 사용가능 -> 높은 복잡성을 가진 분자는 합성 과정이 복잡하거나 비용이 많이 들 수 있음
- 일반적으로 복잡한 구조를 가진 분자는 다양한 생물학적 타겟과 상호작용할 가능성이 높음 -> 이는 분자가 다양한 생물학적 경로에 영향을 미칠 수 있음을 의미
- 복잡한 분자는 종종 특정 생물학적 타겟에 대한 높은 선택성을 가질 수 있음 -> 분자의 고유한 구조적 특성이 특정 수용체의 결합 부위와 잘 맞아떨어져, 높은 특이성을 가진 약물을 설계하는 데 유용
- 복잡한 분자는 대사 과정에서 덜 변형되거나 분해되는 경향 -> 약물의 반감기와 안정성을 높이는데 기여하며, 약물의 효과적인 전달과 장기적인 효능을 유지하는 데 중요

In [20]:
def add_molecular_complexity(df):
    # 분자 복잡성을 계산하는 함수
    def calculate_complexity(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        return GraphDescriptors.BertzCT(molecule) if molecule else 0

    # 분자 복잡성을 계산하여 데이터프레임에 추가
    df['molecular_complexity'] = df['molecule_smiles'].apply(calculate_complexity)

    return df

# 'train' 데이터프레임에 분자 복잡성 특성 추가
train = add_molecular_complexity(train)

In [21]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule,logP,num_hbd,num_hba,num_rotatable_bonds,num_rings,num_c1ccccc1,tpsa,molecular_complexity
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036,3.02098,4,13,10,5,2,180.48,1705.149971
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958,2.97550,4,9,6,4,1,133.29,1182.026982
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502,2.61982,6,11,12,4,1,191.60,1569.180449
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996,4.76192,6,10,10,5,3,178.71,2012.251481
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980,1.33510,5,10,9,3,0,149.61,842.920910


### 화학적 지문
- 화학적 지문은 분자 간의 유사성을 빠르고 효율적으로 비교할 수 있게 해줌. 이를 통해 알려진 활성 분자와 구조적으로 유사한 새로운 분자를 식별하는 데 사용 가능
- 화학적 지문은 대규모 화합물 라이브러리에서 특정 타겟에 대한 잠재적인 활성 분자를 신속하게 스크리닝하는 데 사용(시간, 비용 절약)
- 이진 벡터 형태의 화학적 지문은 기계 학습 모델, 특히 분류 및 회귀 작업에 입력 변수로 사용 가능 -> 이를 통해 분자의 화학적 특성과 생물학적 활성 사이의 관계 모델링 가능
- 화학적 지문을 사용하여 유사한 화학적 특성을 가진 분자들을 그룹화하고, 이를 시각화하여 화학 공간 내에서의 분포 파악 가능

In [22]:
def add_chemical_fingerprint(df):
    # Morgan 지문을 계산하여 비트 벡터 형태로 변환하는 함수
    def calculate_fingerprint(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(molecule, radius=2, nBits=1024)
        return list(fp)

    # 화학적 지문 계산 후, 새로운 특성으로 데이터프레임에 추가
    df['chemical_fingerprint'] = df['molecule_smiles'].apply(calculate_fingerprint)

    return df

# 'train' 데이터프레임에 화학적 지문 특성 추가
train = add_chemical_fingerprint(train)

In [23]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule,logP,num_hbd,num_hba,num_rotatable_bonds,num_rings,num_c1ccccc1,tpsa,molecular_complexity,chemical_fingerprint
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036,3.02098,4,13,10,5,2,180.48,1705.149971,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958,2.97550,4,9,6,4,1,133.29,1182.026982,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502,2.61982,6,11,12,4,1,191.60,1569.180449,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996,4.76192,6,10,10,5,3,178.71,2012.251481,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980,1.33510,5,10,9,3,0,149.61,842.920910,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


### 분자 내 특정 기능적 그룹의 계산
- 각 기능적 그룹은 분자의 화학적 성질과 반응성에 영향 (예: 알코올 그룹은 친수성을 증가시키고, 카복실기는 산성을 제공)
- 특정 기능적 그룹은 생물학적 타겟과의 결합에서 중요한 역할(예: 아민 그룹은 프로테인과 수소 결합을 형성할 수 있고, 암모늄 그룹은 이온 결합을 형성 가능)
- 카복실기와 같은 이온화 가능한 그룹은 용매에 대한 용해도와 분자의 전반적인 투과성 결정에 중요
- 특정 기능적 그룹은 분자의 대사 경로에 영향
- 기능적 그룹의 계산은 약물 설계 과정에서 특정 화학적 특성이나 생물학적 활성과의 관계를 파악하는 데 사용




In [24]:
# SMARTS 패턴을 사용하여 기능적 그룹을 정의
functional_groups = {
    'alcohol': '[OH]',
    'carboxylic_acid': 'C(=O)[OH]',
    'amine': '[NX3;H2,H1,H0;!$(NC=O)]',
    'ketone': '[CX3](=O)[#6]',
    'ammonium': '[NX4+]',
    'ester': 'C(=O)O',
    'nitrile': 'C#N',
    'thiol': '[SH]',
    'sulfide': '[SX2]',
    'sulfoxide': 'S(=O)C',
    'aldehyde': 'C=O',
    'halogen': '[F,Cl,Br,I]'
}

def add_functional_group_counts(df):
    # 각 SMILES에 대해 각 기능적 그룹의 수를 계산하는 함수
    def count_groups(smiles, pattern):
        molecule = Chem.MolFromSmiles(smiles)
        if molecule:
            return len(molecule.GetSubstructMatches(Chem.MolFromSmarts(pattern)))
        return 0

    # 각 기능적 그룹에 대한 계수를 데이터프레임에 추가
    for name, pattern in functional_groups.items():
        df[f'count_{name}'] = df['molecule_smiles'].apply(lambda x: count_groups(x, pattern))

    return df

# 'train' 데이터프레임에 기능적 그룹 계수 특성 추가
train = add_functional_group_counts(train)

In [25]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,...,count_amine,count_ketone,count_ammonium,count_ester,count_nitrile,count_thiol,count_sulfide,count_sulfoxide,count_aldehyde,count_halogen
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,...,3,1,0,0,1,0,0,0,1,0
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,...,3,2,0,0,0,0,1,0,2,4
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,...,3,2,0,0,0,0,0,0,2,1
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,...,4,1,0,0,0,0,0,2,1,3
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,...,3,2,0,0,0,0,1,0,2,0


### 반응성 지표
- 루이스 산 및 루이스 염기는 화학 반응에서 중요 (예: 촉매 반응에서 루이스 산이 촉매로 작용하여 반응 속도 증가)
- 특정 기능적 그룹의 반응성은 약물이 생물학적 타겟과 어떻게 상호작용하는지를 결정하는 데 중요 (예: 루이스 염기성 그룹은 특정 타겟의 금속 이온과 상호작용 가능)
- 분자의 반응성 지표는 화학물질의 안정성 및 잠재적 독성을 평가하는 데 사용 (예: 루이스 산성을 가진 화합물은 특정 조건에서 불안정하거나 반응성이 높을 수 있음)
- 합성 화학에서는 루이스 산 및 염기를 이용하여 원하는 화합물을 효율적으로 합성하는 경로를 설계할 수 있음


In [26]:
# 루이스 산(일반적으로 전자 수용 능력이 높은 그룹)
lewis_acids = {
    'boron_compounds': '[B]',  # 보론 화합물은 전형적인 루이스 산
    'phosphorus_compounds': '[PH2]',  # 포스포루스 화합물
}

# 루이스 염기(일반적으로 전자 공여 능력이 높은 그룹)
lewis_bases = {
    'amines': '[N]',  # 아민
    'ethers': '[O]',  # 에테르
}

def add_reactivity_indicators(df):
    def count_substructures(smiles, pattern):
        mol = Chem.MolFromSmiles(smiles)
        substruct = Chem.MolFromSmarts(pattern)
        return len(mol.GetSubstructMatches(substruct)) if mol else 0

    for name, pattern in {**lewis_acids, **lewis_bases}.items():
        df[f'count_{name}'] = df['molecule_smiles'].apply(lambda x: count_substructures(x, pattern))

    return df

# train 데이터프레임에 반응성 지표 추가
train = add_reactivity_indicators(train)

In [27]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,...,count_nitrile,count_thiol,count_sulfide,count_sulfoxide,count_aldehyde,count_halogen,count_boron_compounds,count_phosphorus_compounds,count_amines,count_ethers
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,...,1,0,0,0,1,0,0,0,5,2
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,...,0,0,1,0,2,4,0,0,5,2
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,...,0,0,0,0,2,1,0,0,5,3
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,...,0,0,0,2,1,3,0,0,5,3
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,...,0,0,1,0,2,0,0,0,4,2


### 치환기 효과(Substituent Effects)
- 치환기 효과를 이해하면 분자의 반응성을 더 정확하게 예측 가능 -> 전자를 끌어당기는 치환기는 분자를 더 반응성이 강하게 만들 수 있고, 전자를 밀어내는 치환기는 반대의 효과를 낼 수 있음
- 약물 분자 설계 시, 치환기 효과를 조절하여 약물의 목표 타겟에 대한 친화성과 선택성 최적화 가능
- 치환기 효과는 분자의 화학적 안정성에 영향 (예: 전자를 끌어당기는 치환기는 산성 환경에서 분자의 안정성을 높일 수 있음)
- 치환기 효과를 통해 분자가 다른 분자 또는 생체 분자와 어떻게 상호작용할지 예측 가능


In [29]:
# 각각의 치환기에 대한 Hammett 상수값을 사전에 정의
hammett_constants = {
    'NO2': 0.78,   # 니트로 그룹
    'CN': 0.66,    # 시아노 그룹
    'COOH': 0.44,  # 카복실산
    'F': 0.34,     # 플루오르
    'Cl': 0.23,    # 클로로
    'Br': 0.23,    # 브로모
    'I': 0.28,     # 요오드
    'OH': -0.37,   # 하이드록시
    'OCH3': -0.27, # 메톡시
    'NH2': -0.66   # 아미노
}

def calculate_substituent_effects(smiles, constants):
    molecule = Chem.MolFromSmiles(smiles)
    effect_sum = 0
    for atom in molecule.GetAtoms():
        if atom.GetAtomMapNum() in constants:  # Atom mapping number를 기준으로 상수 참조
            effect_sum += constants[atom.GetAtomMapNum()]
    return effect_sum

# 각 분자에 대해 치환기 효과 계산
train['substituent_effect'] = train['molecule_smiles'].apply(lambda x: calculate_substituent_effects(x, hammett_constants))

In [30]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,...,count_thiol,count_sulfide,count_sulfoxide,count_aldehyde,count_halogen,count_boron_compounds,count_phosphorus_compounds,count_amines,count_ethers,substituent_effect
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,...,0,0,0,1,0,0,0,5,2,0
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,...,0,1,0,2,4,0,0,5,2,0
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,...,0,0,0,2,1,0,0,5,3,0
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,...,0,0,2,1,3,0,0,5,3,0
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,...,0,1,0,2,0,0,0,4,2,0


### 이성질체 분석
- 스테레오이성질체는 생물학적 환경에서 서로 다르게 작용 가능 (예: 하나의 이성질체는 약물로서 활성을 나타낼 수 있지만, 다른 이성질체는 비활성일 수 있음) -> 이성질체 분석을 통해 약물의 효능과 선택성을 최적화 능
- 특정 이성질체는 독성을 나타낼 수 있어, 이성질체 분석을 통해 약물의 안전성 평가
- 특정 이성질체만을 합성할 수 있는 경로를 개발하는 것은 약물 합성의 중요한 측면


In [31]:
def add_stereoisomers_count(df):
    # 스테레오이성질체의 수를 계산하는 함수
    def calculate_stereoisomers(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        isomers = list(EnumerateStereoisomers.EnumerateStereoisomers(molecule))
        return len(isomers)

    # 스테레오이성질체 수를 계산하여 데이터프레임에 추가
    df['stereoisomers_count'] = df['molecule_smiles'].apply(calculate_stereoisomers)

    return df

# 'train' 데이터프레임에 스테레오이성질체 수 특성 추가
train = add_stereoisomers_count(train)

In [32]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,...,count_sulfide,count_sulfoxide,count_aldehyde,count_halogen,count_boron_compounds,count_phosphorus_compounds,count_amines,count_ethers,substituent_effect,stereoisomers_count
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,...,0,0,1,0,0,0,5,2,0,1
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,...,1,0,2,4,0,0,5,2,0,1
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,...,0,0,2,1,0,0,5,3,0,1
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,...,0,2,1,3,0,0,5,3,0,1
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,...,1,0,2,0,0,0,4,2,0,1


### 공유결합극성
- 분자의 극성은 물리적 성질 예측에 중요 -> 특히 약물의 용해도와 세포막 투과성은 치료 효과와 직결
-  극성 결합은 약물이 생체 분자와의 상호작용, 특히 수소 결합 및 이온 결합에 크게 기여 -> 약물의 효능과 선택성 향상에 중요
- 극성 결합은 약물의 대사 경로와 대사 안정성에 영향 -> 분자의 극성이 높을수록 대사에서 더 활발하게 반응할 가능성이 있음
- 공유결합의 극성은 분자의 화학적 및 물리적 안정성에 영향 -> 극성이 높으면 특정 조건에서 불안정할 수 있음


In [33]:
# 전기음성도 참조 데이터 (Pauling Scale)
electronegativities = {
    'H': 2.20, 'B': 2.04, 'C': 2.55, 'N': 3.04,
    'O': 3.44, 'F': 3.98, 'Si': 1.90, 'P': 2.19,
    'S': 2.58, 'Cl': 3.16, 'Br': 2.96, 'I': 2.66
}

def calculate_bond_polarity(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    polarities = []
    for bond in molecule.GetBonds():
        begin_atom = bond.GetBeginAtom().GetSymbol()
        end_atom = bond.GetEndAtom().GetSymbol()
        if begin_atom in electronegativities and end_atom in electronegativities:
            polarity = abs(electronegativities[begin_atom] - electronegativities[end_atom])
            polarities.append(polarity)
    return sum(polarities) / len(polarities) if polarities else 0

def add_polarity_features(df):
    df['polarity_molecule'] = df['molecule_smiles'].apply(calculate_bond_polarity)
    df['polarity_bb1'] = df['buildingblock1_smiles'].apply(calculate_bond_polarity)
    df['polarity_bb2'] = df['buildingblock2_smiles'].apply(calculate_bond_polarity)
    df['polarity_bb3'] = df['buildingblock3_smiles'].apply(calculate_bond_polarity)
    return df

# 'train' 데이터프레임에 공유결합 극성 변수 추가
train = add_polarity_features(train)

In [34]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,...,count_boron_compounds,count_phosphorus_compounds,count_amines,count_ethers,substituent_effect,stereoisomers_count,polarity_molecule,polarity_bb1,polarity_bb2,polarity_bb3
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,...,0,0,5,2,0,1,0.294545,0.225313,0.210000,0.217778
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,...,0,0,5,2,0,1,0.402973,0.187241,0.345714,0.517500
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,...,0,0,5,3,0,1,0.315581,0.183030,0.256923,0.417500
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,...,0,0,5,3,0,1,0.323191,0.182500,0.265833,0.379286
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,...,0,0,4,2,0,1,0.306333,0.187241,0.240000,0.245000


### 입체화학적 특성 변수
- 키랄 센터를 포함하는 분자는 종종 그들의 거울상 이성질체보다 더 높거나 낮은 생물학적 활성을 보여줌 -> 특정 키랄 분자가 특정 생물학적 타겟에 더 잘맞기 때문
- 입체적 장애가 큰 분자는 종종 높은 선택성을 가질 수 있음 -> 분자가 특정 수용체의 입체적 요구사항에 맞춤화되어 있기 때문
- 키랄 약물은 종종 비키랄 약물보다 대사가 더 예측 가능하며 부작용이 적을 수 있음 -> 키랄 중심이 대사 경로에 영향을 미치기 때문


In [35]:
def add_stereochemical_properties(df):
    # 키랄 센터 수 계산
    def count_chiral_centers(smiles):
        mol = Chem.MolFromSmiles(smiles)
        chiral_centers = [atom for atom in mol.GetAtoms() if atom.GetChiralTag() != Chem.rdchem.ChiralType.CHI_UNSPECIFIED]
        return len(chiral_centers)

    # 이중 결합의 기하학적 이성질체 수 계산
    def count_geometric_isomers(smiles):
        mol = Chem.MolFromSmiles(smiles)
        double_bonds = [bond for bond in mol.GetBonds() if bond.GetBondType() == Chem.rdchem.BondType.DOUBLE]
        geom_isomers = sum(1 for bond in double_bonds if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE)
        return geom_isomers

    df['chiral_centers'] = df['molecule_smiles'].apply(count_chiral_centers)
    df['geometric_isomers'] = df['molecule_smiles'].apply(count_geometric_isomers)

    return df

# 'train' 데이터프레임에 입체화학적 특성 변수 추가
train = add_stereochemical_properties(train)

In [36]:
train.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,...,count_amines,count_ethers,substituent_effect,stereoisomers_count,polarity_molecule,polarity_bb1,polarity_bb2,polarity_bb3,chiral_centers,geometric_isomers
0,38625786,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,...,5,2,0,1,0.294545,0.225313,0.210000,0.217778,0,0
1,131457069,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,...,5,2,0,1,0.402973,0.187241,0.345714,0.517500,2,0
2,257526855,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,...,5,3,0,1,0.315581,0.183030,0.256923,0.417500,1,0
3,46066878,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,...,5,3,0,1,0.323191,0.182500,0.265833,0.379286,0,0
4,77376909,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,...,4,2,0,1,0.306333,0.187241,0.240000,0.245000,0,0


#### id 컬럼제거

In [37]:
train = train.drop('id', axis=1)

In [38]:
train

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,mw_buildingblock1,mw_buildingblock2,mw_buildingblock3,mw_molecule,...,count_amines,count_ethers,substituent_effect,stereoisomers_count,polarity_molecule,polarity_bb1,polarity_bb2,polarity_bb3,chiral_centers,geometric_isomers
0,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1ccc(-n2cncn2)cc1,N#Cc1cccc(N)n1,COc1cccc(C(=O)N[Dy])c1Nc1nc(NCc2ccc(-n3cncn3)c...,BRD4,0,389.407,247.129,119.127,696.036,...,5,2,0,1,0.294545,0.225313,0.210000,0.217778,0,0
1,O=C(N[C@H]1CC[C@@H](C(=O)O)C1)OCC1c2ccccc2-c2c...,NC1=NC(=O)CS1,Nc1ccc(F)c(C(F)(F)F)c1,O=C1CSC(Nc2nc(Nc3ccc(F)c(C(F)(F)F)c3)nc(N[C@H]...,BRD4,0,351.402,116.145,179.116,659.958,...,5,2,0,1,0.402973,0.187241,0.345714,0.517500,2,0
2,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cc1cccnc1NC(=O)CCN.Cl.Cl,Nc1ncc[nH]c1=O,Cc1cccnc1NC(=O)CCNc1nc(Nc2ncc[nH]c2=O)nc(N[C@H...,BRD4,0,421.880,252.145,111.104,725.502,...,5,3,0,1,0.315581,0.183030,0.256923,0.417500,1,0
3,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CS(=O)(=O)Nc1cccc(N)c1,Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1,Cc1c(Br)ccc(C(=O)N[Dy])c1Nc1nc(NCc2nc3c(F)c(F)...,BRD4,0,452.304,186.236,256.083,834.996,...,5,3,0,1,0.323191,0.182500,0.265833,0.379286,0,0
4,O=C(NC1(C(=O)O)CCCC1)OCC1c2ccccc2-c2ccccc21,CC(=O)SCCN.Cl,Nc1cc[nH]n1,CC(=O)SCCNc1nc(Nc2cc[nH]n2)nc(NC2(C(=O)N[Dy])C...,BRD4,0,351.402,155.650,83.094,566.980,...,4,2,0,1,0.306333,0.187241,0.240000,0.245000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65995,CS(=O)(=O)c1ccc(C(=O)O)c(NC(=O)OCC2c3ccccc3-c3...,Cl.NCc1cnc2n1CCOC2,COc1ccncc1CN,COc1ccncc1CNc1nc(NCc2cnc3n2CCOC3)nc(Nc2cc(S(C)...,HSA,1,437.473,189.646,138.170,742.123,...,4,5,0,1,0.356222,0.212059,0.393333,0.325000,0,0
65996,O=C(NC(CC1CCCCC1)C(=O)O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCC1(c2ccncc2)CC1,COC(=O)c1cc(Cl)nc(Cl)c1N,COC(=O)c1cc(Cl)nc(Cl)c1Nc1nc(NCC2(c3ccncc3)CC2...,HSA,1,393.483,221.131,221.043,776.030,...,4,3,0,2,0.285000,0.169688,0.122500,0.412308,0,0
65997,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cl.Cl.NCC=Cc1cccnc1,Cl.NC[C@H]1CC[C@H](C(N)=O)CC1,NC(=O)[C@H]1CC[C@H](CNc2nc(NCC=Cc3cccnc3)nc(N[...,HSA,1,393.464,207.104,192.690,697.170,...,5,2,0,2,0.236098,0.177097,0.147000,0.170000,3,0
65998,Cc1cc(C)c(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)...,Nc1cn[nH]c1,Cc1cc(N)ncn1,Cc1cc(C)c(Nc2nc(Nc3cn[nH]c3)nc(Nc3cc(C)ncn3)n2...,HSA,1,387.435,83.094,109.132,592.956,...,4,1,0,1,0.291429,0.169688,0.245000,0.306250,0,0


In [41]:
train.to_csv("/content/drive/MyDrive/BELKA-DrugPredict/data/train_66000.csv", index=False)